# Методы сбора и обработки данных из сети Интернет
___
___
## Урок 3. Парсинг данных. HTML, Beautiful Soap
___

### Задачи

Необходимо собрать информацию о вакансиях на вводимую должность (используем input или через аргументы получаем должность) с сайтов HH(обязательно) и/или Superjob(по желанию). Приложение должно анализировать несколько страниц сайта (также вводим через input или аргументы). Получившийся список должен содержать в себе минимум:
Наименование вакансии.
Предлагаемую зарплату (разносим в три поля: минимальная и максимальная и валюта. цифры преобразуем к цифрам).
Ссылку на саму вакансию.
Сайт, откуда собрана вакансия.

In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [2]:
USER_AGENT = 'Mozilla/5.0 (Macintosh; Intel Mac OS X 13_2) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36'
URL1 = 'https://hh.ru/search/vacancy?text=Datascience&from=suggest_post&salary=&area=1&ored_clusters=true&enable_snippets=true'
URL = 'https://hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=Data Science&no_magic=true&L_save_area=true&items_on_page=20'

In [3]:
headers = {
    'User-Agent': USER_AGENT,
}

In [4]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [5]:
result = parse_hh(URL, headers)

Cтраница №1, ссылка: https://hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=Data%20Science&no_magic=true&L_save_area=true&items_on_page=20
Cтраница №2, ссылка: https://hh.ru/search/vacancy?area=88&search_field=name&search_field=company_name&search_field=description&text=Data+Science&no_magic=true&L_save_area=true&items_on_page=20&page=1&hhtmFrom=vacancy_search_list
Парсинг завершен


In [6]:
len(result)

30

In [7]:
result

[{'vacancy_name': 'Специалист по Data Science',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://hh.ru/vacancy/77484387?from=vacancy_search_list&query=Data+Science',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'IT Recriuter ML, AI, Big Data',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://hh.ru/vacancy/77618077?from=vacancy_search_list&query=Data+Science',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Python backend-разработчик в команду Data Science',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://hh.ru/vacancy/77050357?from=vacancy_search_list&query=Data+Science',
  'vacancy_source': 'hh.ru'},
 {'vacancy_name': 'Системный аналитик (systems analyst)',
  'vacancy_salary': 'от 120\u202f000 руб.',
  'min_salary': 120000,
  'max_salary': None,
  'currency_s

In [8]:
data = pd.DataFrame(result)
data

,vacancy_name,vacancy_salary,min_salary,max_salary,currency_salary,vacancy_link,vacancy_source
0,Специалист по Data Science,None,NaN,NaN,None,https://hh.ru/vacancy/77484387?from=vacancy_se...,hh.ru
1,"IT Recriuter ML, AI, Big Data",None,NaN,NaN,None,https://hh.ru/vacancy/77618077?from=vacancy_se...,hh.ru
2,Python backend-разработчик в команду Data Science,None,NaN,NaN,None,https://hh.ru/vacancy/77050357?from=vacancy_se...,hh.ru
3,Системный аналитик (systems analyst),от 120 000 руб.,120000.0,NaN,руб.,https://hh.ru/vacancy/77145441?from=vacancy_se...,hh.ru
4,Инженер - исследователь (Data Science),None,NaN,NaN,None,https://hh.ru/vacancy/77129293?from=vacancy_se...,hh.ru
5,Assistant Professor - Data Science&AI (eng),None,NaN,NaN,None,https://hh.ru/vacancy/73328580?from=vacancy_se...,hh.ru
6,Associate Professor - Data Science&AI (eng),None,NaN,NaN,None,https://hh.ru/vacancy/73328887?from=vacancy_se...,hh.ru
7,Системный аналитик (Инвестиционный бизнес),None,NaN,NaN,None,https://hh.ru/vacancy/77698888?from=vacancy_se...,hh.ru
8,"Инженер данных (качество сервисов,SLA)",None,NaN,NaN,None,https://hh.ru/vacancy/70972873?from=vacancy_se...,hh.ru
9,Специалист технической поддержки (чаты),от 40 000 руб.,40000.0,NaN,руб.,https://hh.ru/vacancy/77127896?from=vacancy_se...,hh.ru
